<a href="https://colab.research.google.com/github/thiswind/machine-learning-study/blob/main/gan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

复现CSDN文章[【实战】kaggle猫狗大战-卷积神经网络实现猫狗识别](https://blog.csdn.net/Mind_programmonkey/article/details/99309528)

In [1]:
!nvidia-smi
!pwd

Wed Nov  9 16:52:07 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    43W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
!ln -s drive/MyDrive/colab $(pwd)/colab
!mkdir -p ./colab/gan
DRIVE_DIR = './colab/gan/'
!ls -l
!ls -l ./colab/gan

Mounted at /content/drive/
total 8
lrwxrwxrwx 1 root root   19 Nov  9 16:52 colab -> drive/MyDrive/colab
drwx------ 5 root root 4096 Nov  9 16:52 drive
drwxr-xr-x 1 root root 4096 Nov  7 14:37 sample_data
total 0


In [3]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle
!echo '{"username":"windthis","key":"a968dadd8de108941769a83c92588874"}' > ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!rm -f *.zip
!rm -f *.csv
!rm -rf ./train/
!rm -rf ./test/
!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition
!unzip -q dogs-vs-cats-redux-kernels-edition.zip
!unzip -q ./train.zip
!unzip -q ./test.zip

 97% 787M/814M [00:05<00:00, 191MB/s]
100% 814M/814M [00:05<00:00, 145MB/s]


In [4]:
# 导入开发需要的库
import keras
import os
import shutil
import threading
import numpy as np
import cv2
import h5py

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from keras.models import *
from keras.layers import *

from keras.applications import *
from keras.preprocessing import *
from keras.preprocessing.image import *


In [5]:
def load_data(load_type="train"):
    path = None
    n = 25000
    if load_type=="train":
        imgs = []
        labels = []
        
        path = "./train/"
        img_names = os.listdir(path)
        
        for name in img_names:
            imgs.append("train/"+name)
            labels.append([0] if name[:3] == "cat" else [1])
            
        train_img_names,valid_img_names,train_labels,valid_labels = train_test_split(imgs,labels,test_size=0.2,random_state=42)
        return train_img_names,valid_img_names,train_labels,valid_labels
    else:
        # test,don`t have the labels
        path = "./test"
        img_names = os.listdir(path)
        imgs = []
        for img in img_names:
            imgs.append(img)
                
        return imgs


In [6]:
train_img_names,valid_img_names,train_labels,valid_labels = load_data()

In [7]:
print(train_img_names[:5])
print(train_labels[:5])
print(len(train_img_names)+len(valid_img_names))

['train/dog.2474.jpg', 'train/cat.11618.jpg', 'train/cat.12202.jpg', 'train/dog.4031.jpg', 'train/cat.5840.jpg']
[[1], [0], [0], [1], [0]]
25000


In [8]:
# 定义keras的generator方法
# custom keras generator
class MOGenerator(keras.utils.Sequence):
    def __init__(self, data, n, des_size=(224, 224), means=None, stds=None,
                 is_directory=True, batch_size=32, shuffle=True, seed=0):
        '''
        data: tuple of (x,y)
        n: data size
        des_size: standard size
        means: the dataset mean of RGB,default is imagenet means [103.939, 116.779, 123.68]
        batch_size: default is 32
        shuffle: random the data,default is True
        '''
        self.x = np.array(data[0])
        if len(data) >= 2:
            self.y = np.array(data[1])
        else:
            self.y = None
        self.n = n
        self.des_size = des_size
        self.is_directory = is_directory
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.lock = threading.Lock()
        self.index_array = self._set_index_array()
        self.means = means
        self.stds = stds

    def reset_index(self):
        self.batch_index = 0

    def _set_index_array(self):
        self.index_array = np.arange(self.n)
        if self.shuffle:
            np.random.shuffle(self.index_array)

    def on_epoch_end(self):
        # 重置索引数组
        self._set_index_array()

    def __len__(self):
        # 计算batch的总数量
        return int(np.ceil(self.n / self.batch_size))

    def __getitem__(self, idx):
        if self.index_array is None:
            self._set_index_array()
        index_array = self.index_array[self.batch_size * idx:
                                       self.batch_size * (idx + 1)]
        return self._data_generate(index_array)

    def _data_generate(self, index_array):
        # read from path
        # request the memory
        imgs = np.zeros((len(index_array), self.des_size[0], self.des_size[1], 3), dtype=np.uint8)
        # read the data
        if self.is_directory:
            img_names = self.x[index_array]
            for name_index in range(len(img_names)): 
                img = cv2.imread(img_names[name_index])
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    img = cv2.resize(img, self.des_size)
                    imgs[name_index] = img
        else:
            for i in range(len(index_array)):
                img = self.x[index_array[i]]
                img = cv2.resize(img, self.des_size)
                imgs[i] = img
        if self.y is not None:
            labels = self.y[index_array]
        if labels is None:
            return imgs
        else:
            return imgs, labels



In [9]:
batch_size = 32

train_img_names,valid_img_names,train_labels,valid_labels = load_data()

test_img_names = load_data(load_type="test")

train_generator_224 = MOGenerator((train_img_names,train_labels), len(train_img_names), des_size=(224,224),
                                  batch_size=batch_size, shuffle=True)

train_generator_299 = MOGenerator((train_img_names,train_labels), len(train_img_names), des_size=(299,299),
                                  batch_size=batch_size, shuffle=True)

valid_generator_299 = MOGenerator((valid_img_names,valid_labels), len(valid_img_names), des_size=(299,299),
                                  batch_size=batch_size, shuffle=False)

valid_generator_224 = MOGenerator((valid_img_names,valid_labels), len(valid_img_names), des_size=(224,224),
                                  batch_size=batch_size, shuffle=True)

test_generator_299 = MOGenerator((test_img_names), len(test_img_names), des_size=(299,299),
                                  batch_size=batch_size, shuffle=False)

test_generator_224 = MOGenerator((test_img_names), len(test_img_names), des_size=(224,224),
                                  batch_size=batch_size, shuffle=False)


In [10]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import resnet50

In [11]:
# 加载Resnet50网络, Include_top: 是否包含卷积之后的全连接层.
base_model = ResNet50(input_tensor=Lambda(resnet50.preprocess_input)(Input(shape=(224,224,3))), 
                      weights="imagenet", include_top=False)
# 遍历所有层, 将预训练模型的卷积层设置为不可训练, 这样它们的参数就不会被改变了.
for layers in base_model.layers:
    layers.trainable = False

# 重新设置输出层, 我们的类别是两类, 只需要一个神经元即可.
x = GlobalAveragePooling2D()(base_model.output)
x = Dropout(0.25)(x)
x = Dense(1, activation="sigmoid")(x)

# 实例化模型
model = Model(base_model.input, x)

# 设置优化器, 损失函数, 展示参数信息
model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

# 开始训练, 通过设置迭代器模式
model.fit_generator(train_generator_224,len(train_img_names)//batch_size,epochs=5,
                    validation_data=valid_generator_224,validation_steps=len(valid_img_names)//batch_size,shuffle=False)


94765736/94765736 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/5
625/625 [==============================] - 80s 108ms/step - loss: 0.0620 - accuracy: 0.9779 - val_loss: 0.0386 - val_accuracy: 0.9856
Epoch 2/5
625/625 [==============================] - 65s 105ms/step - loss: 0.0318 - accuracy: 0.9881 - val_loss: 0.0452 - val_accuracy: 0.9836
Epoch 3/5
625/625 [==============================] - 66s 105ms/step - loss: 0.0274 - accuracy: 0.9898 - val_loss: 0.0348 - val_accuracy: 0.9876
Epoch 4/5
625/625 [==============================] - 65s 104ms/step - loss: 0.0272 - accuracy: 0.9906 - val_loss: 0.0350 - val_accuracy: 0.9874
Epoch 5/5
625/625 [==============================] - 66s 105ms/step - loss: 0.0247 - accuracy: 0.9917 - val_loss: 0.0373 - val_accuracy: 0.9868


In [12]:
## inception
inception = inception_v3.InceptionV3(include_top=False,
        weights="imagenet",input_tensor=Lambda(inception_v3.preprocess_input)(Input(shape=(299,299,3))),pooling="avg")
output = inception.output

output = Dropout(0.25)(output)

prediction = Dense(1,activation="sigmoid")(output)

inception_model = Model(inputs=inception.input,outputs=prediction)

for layer in inception.layers: #[:-res_global_pool_index]
    layer.trainable = False

inception_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

inception_model.fit_generator(train_generator_299,(len(train_img_names) + batch_size - 1)//batch_size,epochs=5,
                       validation_data=valid_generator_299,validation_steps=len(valid_img_names)//batch_size)


87910968/87910968 [==============================] - 0s 0us/step
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


625/625 [==============================] - 77s 116ms/step - loss: 0.0725 - accuracy: 0.9782 - val_loss: 0.0274 - val_accuracy: 0.9928
Epoch 2/5
625/625 [==============================] - 71s 113ms/step - loss: 0.0237 - accuracy: 0.9927 - val_loss: 0.0226 - val_accuracy: 0.9930
Epoch 3/5
625/625 [==============================] - 70s 113ms/step - loss: 0.0207 - accuracy: 0.9942 - val_loss: 0.0213 - val_accuracy: 0.9932
Epoch 4/5
625/625 [==============================] - 70s 112ms/step - loss: 0.0183 - accuracy: 0.9943 - val_loss: 0.0219 - val_accuracy: 0.9928
Epoch 5/5
625/625 [==============================] - 70s 113ms/step - loss: 0.0180 - accuracy: 0.9941 - val_loss: 0.0214 - val_accuracy: 0.9934


In [13]:
## xception
xcep = Xception(include_top=False, weights="imagenet", 
                input_tensor=Lambda(xception.preprocess_input)(Input(shape=(299,299,3))),pooling="avg")
output = xcep.output

output = Dropout(0.25)(output)

prediction = Dense(1,activation="sigmoid")(output)

xcep_model = Model(inputs=xcep.input,outputs=prediction)

for layer in xcep.layers:
    layer.trainable = False

xcep_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

xcep_model.fit_generator(train_generator_299,(len(train_img_names) + batch_size - 1)//batch_size,epochs=5,
                       validation_data=valid_generator_299,validation_steps=len(valid_img_names)//batch_size)


83683744/83683744 [==============================] - 0s 0us/step
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


625/625 [==============================] - 76s 118ms/step - loss: 0.0680 - accuracy: 0.9864 - val_loss: 0.0281 - val_accuracy: 0.9926
Epoch 2/5
625/625 [==============================] - 70s 112ms/step - loss: 0.0219 - accuracy: 0.9939 - val_loss: 0.0234 - val_accuracy: 0.9928
Epoch 3/5
625/625 [==============================] - 70s 112ms/step - loss: 0.0179 - accuracy: 0.9948 - val_loss: 0.0207 - val_accuracy: 0.9926
Epoch 4/5
625/625 [==============================] - 70s 112ms/step - loss: 0.0167 - accuracy: 0.9947 - val_loss: 0.0199 - val_accuracy: 0.9930
Epoch 5/5
625/625 [==============================] - 70s 112ms/step - loss: 0.0154 - accuracy: 0.9952 - val_loss: 0.0192 - val_accuracy: 0.9936


In [14]:
model.save_weights("resnet.h5")
model.save_weights("xcep.h5")
model.save_weights("incep.h5")


In [15]:
from keras.preprocessing.image import *

# 获取所有训练集的图片名
train_filenames = os.listdir("train")
test_filenames = os.listdir("test")

# 文件名是特殊命名形式的, 所以直接获取分类后的文件
train_cat = list(filter(lambda x:x[:3] == 'cat', train_filenames))
train_dog = list(filter(lambda x:x[:3] == 'dog', train_filenames))
def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

# 创建目录
rmrf_mkdir('train2')
os.mkdir('train2/cat')
os.mkdir('train2/dog')

rmrf_mkdir('valid2')
os.mkdir('valid2/cat')
os.mkdir('valid2/dog')

rmrf_mkdir('test2')
os.mkdir("test2/test")
for filename in test_filenames:
  # 建立软链（不会从创建文件, 有点类似快捷方式）
  os.symlink("/content/test/"+filename, "/content/test2/test/"+filename)

for filename in train_cat[:-2500]:
  os.symlink("/content/train/"+filename, "/content/train2/cat/"+filename)

for filename in train_dog[:-2500]:
  os.symlink("/content/train/"+filename, "/content/train2/dog/"+filename)

for filename in train_cat[-2500:]:
  os.symlink("/content/train/"+filename, "/content/valid2/cat/"+filename)
    
for filename in train_dog[-2500:]:
  os.symlink("/content/train/"+filename, "/content/valid2/dog/"+filename)
    
# 使用Keras默认的生成器, 因为我们已经生成对应的目录了
# 224 是 resnet的处理方式, 299是xception, inception的处理大小
gen = ImageDataGenerator()

train_generator_224 = gen.flow_from_directory("train2", (224,224), shuffle=False, 
                                              batch_size=batch_size,class_mode='binary')

valid_generator_224 = gen.flow_from_directory("valid2", (224,224), shuffle=False, 
                                              batch_size=batch_size,class_mode='binary')

test_generator_224 = gen.flow_from_directory("test2", (224,224), shuffle=False, 
                                             batch_size=batch_size, class_mode=None)

train_generator_299 = gen.flow_from_directory("train2", (299,299), shuffle=False, 
                                              batch_size=batch_size,class_mode='binary')

valid_generator_299 = gen.flow_from_directory("valid2", (299,299), shuffle=False, 
                                              batch_size=batch_size,class_mode='binary')

test_generator_299 = gen.flow_from_directory("test2", (299,299), shuffle=False, 
                                             batch_size=batch_size, class_mode=None)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.
Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [16]:
import h5py

# 特征向量的保存方法
def write_feature(model_name,model,train_generator,train_labels,valid_generator,valid_labels,test_generator,batch_size=32):
    # 通过模型名字来加载不同的权重
    if model_name == 'resnet_feature':
        model.load_weights('resnet.h5',by_name=True)
    elif model_name == 'inception_feature':
        model.load_weights('incep.h5',by_name=True)
    else:
        model.load_weights('xcep.h5',by_name=True)
    # 转换为numpy数组
    train_labels = np.array(train_labels)
    valid_labels = np.array(valid_labels)
    # 直接进行输出, 得到特征向量
    train_feature = model.predict_generator(train_generator,int(np.ceil(train_generator.samples/batch_size)),verbose=1)
    valid_feature = model.predict_generator(valid_generator,int(np.ceil(valid_generator.samples/batch_size)),verbose=1)
    test_feature  = model.predict_generator(test_generator,int(np.ceil(test_generator.samples/batch_size)),verbose=1)
    print("train_feature.shape:",train_feature.shape)
    print("valid_feature.shape:",valid_feature.shape)
    # 保存到本地
    with h5py.File(model_name+'.h5','w') as file:
        file.create_dataset("train",data=train_feature,dtype="float32")
        file.create_dataset('trian_labels',data=np.array(train_generator.classes),dtype="uint8")
        file.create_dataset("valid",data=valid_feature,dtype="float32")
        file.create_dataset("valid_labels",data=np.array(valid_generator.classes),dtype="uint8")
        file.create_dataset("test",data=test_feature,dtype="float32")

        


In [17]:
# resnet50
write_feature('resnet_feature',Model(inputs=model.input,outputs=model.layers[-3].output),
              train_generator_224,train_labels,valid_generator_224,valid_labels,test_generator_224)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  app.launch_new_instance()


625/625 [==============================] - 59s 94ms/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


157/157 [==============================] - 15s 93ms/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


391/391 [==============================] - 38s 94ms/step
train_feature.shape: (20000, 2048)
valid_feature.shape: (5000, 2048)


In [18]:
# inception
write_feature('inception_feature',Model(inputs=inception_model.input,outputs=inception_model.layers[-3].output),
              train_generator_299,train_labels, valid_generator_299,valid_labels,test_generator_299)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  app.launch_new_instance()


625/625 [==============================] - 69s 108ms/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


157/157 [==============================] - 17s 111ms/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


391/391 [==============================] - 44s 109ms/step
train_feature.shape: (20000, 2048)
valid_feature.shape: (5000, 2048)


In [19]:
# xception
write_feature('xception_feature',Model(inputs=xcep_model.input,outputs=xcep_model.layers[-3].output),
              train_generator_299,train_labels,valid_generator_299,valid_labels,test_generator_299)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  app.launch_new_instance()


625/625 [==============================] - 68s 108ms/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


157/157 [==============================] - 17s 109ms/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


391/391 [==============================] - 43s 108ms/step
train_feature.shape: (20000, 2048)
valid_feature.shape: (5000, 2048)


In [20]:
feature_files = ['resnet_feature.h5','inception_feature.h5','xception_feature.h5']

X_train = []
y_train = []
X_valid = []
y_valid = []
X_test = []

for file_name in feature_files:
    with h5py.File(file_name, 'r') as h:
        X_train.append(np.array(h['train']))
        X_valid.append(np.array(h['valid']))
        X_test.append(np.array(h['test']))
        y_train = np.array(h['trian_labels'])
        y_valid = np.array(h['valid_labels'])
        print(np.array(h['train']).shape,np.array(h['valid']).shape,np.array(h['test']).shape)

X_train = np.concatenate(X_train, axis=1)
X_valid = np.concatenate(X_valid, axis=1)
X_test = np.concatenate(X_test, axis=1)

print("last:",X_train.shape,X_valid.shape,X_test.shape)


(20000, 2048) (5000, 2048) (12500, 2048)
(20000, 2048) (5000, 2048) (12500, 2048)
(20000, 2048) (5000, 2048) (12500, 2048)
last: (20000, 6144) (5000, 6144) (12500, 6144)


In [21]:
from sklearn.utils import shuffle
# 打乱数据，模拟现实中的随机效果
X_train, y_train = shuffle(X_train, y_train)


In [22]:
import keras.utils

input_tensor = Input(X_train.shape[1:])
x = input_tensor
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
concatenate_model = Model(inputs=input_tensor, outputs=x)

concatenate_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

concatenate_model.fit(X_train,y_train,batch_size=128, epochs=5,validation_data=(X_valid,y_valid))#validation_split=0.2


Epoch 1/5
157/157 [==============================] - 1s 5ms/step - loss: 0.0676 - accuracy: 0.9743 - val_loss: 0.0209 - val_accuracy: 0.9942
Epoch 2/5
157/157 [==============================] - 1s 4ms/step - loss: 0.0196 - accuracy: 0.9938 - val_loss: 0.0186 - val_accuracy: 0.9938
Epoch 3/5
157/157 [==============================] - 1s 4ms/step - loss: 0.0162 - accuracy: 0.9945 - val_loss: 0.0238 - val_accuracy: 0.9916
Epoch 4/5
157/157 [==============================] - 1s 4ms/step - loss: 0.0148 - accuracy: 0.9945 - val_loss: 0.0181 - val_accuracy: 0.9944
Epoch 5/5
157/157 [==============================] - 1s 4ms/step - loss: 0.0134 - accuracy: 0.9959 - val_loss: 0.0175 - val_accuracy: 0.9940


In [66]:
import pandas as pd

y_pred = concatenate_model.predict(X_test, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

df = pd.read_csv("sample_submission.csv")

image_size = (224, 224)
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("/content/test2/", image_size, shuffle=False, 
                                         batch_size=16, class_mode=None)
for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    # df.set_value(index-1, 'label', y_pred[i])
    df.at[index-1, 'label'] = y_pred[i]
    break

df.to_csv('sample_submission_try.csv', index=False)


print(df.head(20))


391/391 [==============================] - 1s 1ms/step
Found 12500 images belonging to 1 classes.
    id  label
0    1  0.995
1    2  0.500
2    3  0.500
3    4  0.500
4    5  0.500
5    6  0.500
6    7  0.500
7    8  0.500
8    9  0.500
9   10  0.500
10  11  0.500
11  12  0.500
12  13  0.500
13  14  0.500
14  15  0.500
15  16  0.500
16  17  0.500
17  18  0.500
18  19  0.500
19  20  0.500


In [67]:
!kaggle competitions submit -c dogs-vs-cats-redux-kernels-edition -f sample_submission_try.csv -m "make a try"

100% 111k/111k [00:00<00:00, 592kB/s]
Successfully submitted to Dogs vs. Cats Redux: Kernels Edition